In [1]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LANGCHAIN_API_KEY"] = "..."
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv(Path("../.env.test"), override=True)

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import json

from zenbase.types import LMRequest, deflm
from langsmith import traceable
from langsmith.schemas import Run, Example
from langsmith.wrappers import wrap_openai
from openai import OpenAI

openai = wrap_openai(OpenAI())

# Define your LLM function
@traceable
def openai_json_response(inputs: dict) -> dict:
    messages = [
        {
            "role": "system",
            "content": "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples. Think step by step. Respond with a JSON object.",
        },
        {"role": "user", "content": json.dumps(inputs)}
    ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    return json.loads(response.choices[0].message.content)

# Define your Langsmith evaluator
def score_answer(run: Run, example: Example):
    output = run.outputs["answer"]
    target = example.outputs["answer"]
    return {
        "key": "correctness",
        "score": int(output == target),
    }

In [4]:
# Evaluate using LangSmith
from langsmith import Client, evaluate

langsmith = Client()
evalset = list(langsmith.list_examples(dataset_name="gsm8k-test-examples"))

evaluate_kwargs = dict(
    data=evalset,
    evaluators=[score_answer],
    client=langsmith,
    max_concurrency=2,
)

evaluate(openai_json_response, **evaluate_kwargs)

View the evaluation results for experiment: 'yellow-rain-65' at:
https://smith.langchain.com/o/b0308fb6-cdef-5df3-affa-b8dba287e3ed/datasets/1b7abb1a-8922-4eba-b0b6-b617241d8794/compare?selectedSessions=7acd419e-56bd-4382-8190-1671ce97f802




0it [00:00, ?it/s]

Mathing...Mathing...

Mathing...
Mathing...
Mathing...
{'answer': 540}
{'answer': 60000}
{'answer': 80}
{'answer': 3}
{'answer': '50'}


<ExperimentResults yellow-rain-65>

In [5]:
# Wrap your existing chain with @deflm and take in a `LMRequest` object
# An LMRequest has the inputs for your chain and has a `zenbase` attribute.
# This `zenbase` attribute includes the fields that Zenbase optimises. 
@deflm
@traceable
def openai_json_response(request: LMRequest) -> dict:
    messages = [
        {
            "role": "system",
            "content": "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples. Think step by step. Respond with a JSON object.",
        },
    ]

    for demo in request.zenbase.demos:
        messages += [
            {"role": "user", "content": json.dumps(demo.inputs)},
            {"role": "assistant", "content": json.dumps(demo.outputs)},
        ]
    messages.append({"role": "user", "content": json.dumps(request.inputs)})

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    return json.loads(response.choices[0].message.content)

In [6]:
from zenbase.helpers.langchain import ZenLangSmith
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot

demoset = ZenLangSmith.examples_to_demos(
    langsmith.list_examples(dataset_name="gsm8k-golden-demos")
)
optimizer = LabeledFewShot(demoset=demoset, shots=3)

best_fn, candidates = optimizer.perform(
    # Pass deflm decorated function
    openai_json_response,
    # Exactly the same as what you are passing to your evaluate function
    evaluator=ZenLangSmith.metric_evaluator(**evaluate_kwargs),
    samples=2,
    rounds=1,
)

View the evaluation results for experiment: 'zenbase-mandatory-incremental-migration-37539b3e' at:
https://smith.langchain.com/o/b0308fb6-cdef-5df3-affa-b8dba287e3ed/datasets/1b7abb1a-8922-4eba-b0b6-b617241d8794/compare?selectedSessions=330837a5-d58a-4075-9dbe-82d6f26ef36d




0it [00:00, ?it/s]

Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
{'answer': 'He runs 60 * 3 = <<60*3=180>>180 meters in total every day.\nSo, he runs 180 * 3 = <<180*3=540>>540 meters a week.\n#### 540'}{'answer': "The increase in value is 150% of the original cost + repairs, which is 150% of (80,000 + 50,000) = 1.5 * 130,000 = $<<1.5*130000=195000>>195,000.\nJosh's profit is the new value minus the total cost, which is 195,000 - 80,000 - 50,000 = $<<195000-80000-50000=65000>>65,000.\n#### 65000"}

{'answer': "Janet uses 3+4 = <<3+4=7>>7 eggs every day.\nShe has 16-7 = <<16-7=9>>9 eggs remaining to sell.\nTherefore, she makes 9 * 2 = $<<9*2=18>>18 every day at the farmers' market.\n#### 18"}
{'answer': 'A robe takes 2 bolts of blue fiber and 2/2=1 bolt of white fiber.\nIn total, it takes 2+1=<<2+1=3>>3 bolts.\n#### 3'}
{'answer': 'In total, Wendi gives 15 + 25 = <<15+25=40>>40 cups of feed in the first two meals.\nEach chicken consumes 3 * 3 = <<3*3=9>>9 cups of feed per day.\nSo, the flock of 2

0it [00:00, ?it/s]

Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
{'answer': 'The total cost of the house and repairs is 80000 + 50000 = $<<80000+50000=130000>>130000\nThe increased value of the house is 130000 * 150% = $<<130000*150/100=195000>>195000\nTherefore, the profit he made is 195000 - 130000 = $<<195000-130000=65000>>65000\n#### 65000'}{'answer': 'He runs a total of 3 sprints x 60 meters = <<3*60=180>>180 meters per day.\nSo in a week, he runs 180 meters x 3 times a week = <<180*3=540>>540 meters.\n#### 540'}

{'answer': 'It takes 2 / 2 = <<2/2=1>>1 bolt of white fiber.\nSo, in total it takes 2 + 1 = <<2+1=3>>3 bolts.\n#### 3'}
{'answer': "Janet uses 3 + 4 = <<3+4=7>>7 eggs per day.\nThe number of eggs she sells at the market daily is 16 - 7 = <<16-7=9>>9 eggs.\nTherefore, she makes 9 x 2 = <<9*2=18>>18 dollars every day at the farmers' market.\n#### 18"}
{'answer': 'In total, Wendi gives her chickens 15 + 25 = <<15+25=40>>40 cups of feed in the morning and afternoon.\nEach chicken rece

In [7]:
# Now you can use your zenbase fn
best_fn({"question": "If I have 30% of shares, and Mo has 24.5% of shares, how many of our 10M shares are unassigned?"})

Mathing...


{'answer': 'The total percentage of assigned shares is 30% + 24.5% = 54.5%.\nThis means the percentage of unassigned shares is 100% - 54.5% = 45.5%.\nTherefore, the number of unassigned shares is 10,000,000 * (45.5/100) = 4,550,000.\n#### 4550000'}

In [8]:
# You can also save the zenbase params for re-use
import pickle

pickled_zenbase = pickle.dumps(best_fn.zenbase)
openai_json_response.zenbase = pickle.loads(pickled_zenbase)

openai_json_response({"question": "What is 2 + 2?"}) # uses the best few-shot demos

Mathing...


{'answer': '4'}